In [1]:
from data import Data
import numpy as np

In [2]:
d = Data()
X_train,y_train = d.get_10()
X_cross, y_cross = d.get_dev()
X_test,y_test = d.get_test()

loaded 100
loaded 200
loaded 300
loaded 400
loaded 500
loaded 600
loaded 700
loaded 800
loaded 900
loaded 1000
loaded 1100
loaded 1200
loaded 1300
loaded 1400
loaded 1500
loaded 1600
loaded 1700
loaded 1800
loaded 1900
loaded 2000
loaded 2100
loaded 2200
loaded 2300
loaded 2400
loaded 2500
loaded 2600
loaded 2700
loaded 2800
loaded 2900
loaded 3000
loaded 3100
loaded 3200
loaded 3300
loaded 3400
loaded 3500
loaded 3600
loaded 3700
loaded 3800
loaded 3900
loaded 4000
loaded 4100
loaded 4200
loaded 4300
loaded 4400
loaded 4500
loaded 4600
loaded 4700
loaded 4800
loaded 4900
loaded 5000
loaded 5100
loaded 5200
loaded 5300
loaded 5400
loaded 5500
loaded 5600
loaded 5700
loaded 5800
loaded 5900
loaded 6000
loaded 6100
loaded 6200
loaded 6300
loaded 6400
loaded 6500
loaded 6600
loaded 6700
loaded 6800
loaded 6900
loaded 7000
loaded 7100
loaded 7200
loaded 7300
loaded 7400
loaded 7500
loaded 7600
loaded 7700
loaded 7800
loaded 7900
loaded 8000
loaded 8100
loaded 8200
loaded 8300
loaded 8400
l

In [3]:
X_train = X_train.transpose(0, 3, 1, 2)
y_train = y_train
print(X_train.shape)
print(y_train.shape)
print(X_cross.shape)
print(y_cross.shape)
print(X_test.shape)
print(y_test.shape)

(742, 3, 96, 128)
(742,)
(3230, 96, 128, 3)
(3230,)
(2423, 96, 128, 3)
(2423,)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [5]:
class MyCustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        img = self.X[index]
        label = self.y[index]
        return (img, label)

    def __len__(self):
        return self.X.shape[0]

In [6]:
NUM_TRAIN = X_train.shape[0]
train_dataset = MyCustomDataset(X_train, y_train)
loader_train = DataLoader(train_dataset, batch_size=10,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

test_dataset = MyCustomDataset(X_test, y_test)
loader_test = DataLoader(test_dataset, batch_size=10)


In [7]:
def flatten(X):
    N = X.shape[0]
    return X.view(N,-1)

In [8]:
def check_accuracy(loader, model): 
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            scores = model(Variable(x.float()))
            _, preds = scores.max(1)
            num_correct += (preds == Variable(y.long())).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [12]:
print_every = 10
def train(model, optimizer, epochs=6):
    loss_arr = []
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()
            scores = model(Variable(x.float()))
            loss = F.cross_entropy(scores, Variable(y.long()))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss))
                loss_arr += [loss.item()]
                check_accuracy(loader_train, model)
    return loss_arr

In [13]:
class Flatten(nn.Module):
    def forward(self, x): 
        return flatten(x)

In [14]:
# hidden_layer_size = 500
channel_1 = 16
channel_2 = 32
learning_rate = 5e-5
num_classes = 10

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 5, padding=2),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2 * 96 * 128, 10)
) 
optimizer = optim.Adam(model.parameters(),lr = learning_rate)
loss_arr = train(model,optimizer)
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.plot(loss_arr)
plt.show()

Iteration 0, loss = 31.5115
Got 107 / 742 correct (14.42)
Iteration 10, loss = 465.3200
Got 81 / 742 correct (10.92)
Iteration 20, loss = 140.3864
Got 146 / 742 correct (19.68)
Iteration 30, loss = 113.5323
Got 104 / 742 correct (14.02)
Iteration 40, loss = 133.1652
Got 128 / 742 correct (17.25)
Iteration 50, loss = 45.0490
Got 169 / 742 correct (22.78)
Iteration 60, loss = 87.5076
Got 199 / 742 correct (26.82)
Iteration 70, loss = 54.5097
Got 253 / 742 correct (34.10)
Iteration 0, loss = 39.4796
Got 226 / 742 correct (30.46)
Iteration 10, loss = 46.5419
Got 265 / 742 correct (35.71)
Iteration 20, loss = 36.7251
Got 308 / 742 correct (41.51)
Iteration 30, loss = 7.5977
Got 355 / 742 correct (47.84)
Iteration 40, loss = 9.8450
Got 355 / 742 correct (47.84)
Iteration 50, loss = 8.5891
Got 442 / 742 correct (59.57)
Iteration 60, loss = 14.2191
Got 490 / 742 correct (66.04)
Iteration 70, loss = 25.2922
Got 491 / 742 correct (66.17)
Iteration 0, loss = 12.3611
Got 307 / 742 correct (41.37)


NameError: name 'plt' is not defined